<a href="https://colab.research.google.com/github/KNUckle-llm/experiments/blob/main/saveVectorDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install PyPDF2
!pip install -q sentence-transformers
!pip install -q chromadb sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 99.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 77.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 35.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 86.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.5 MB/s eta 0:00:00
  Installing build dependencies ... done
 

In [ ]:
import re
import chromadb
import os
import unicodedata
from chromadb.config import Settings
from datetime import date
from chromadb import PersistentClient
from sentence_transformers import SentenceTransformer
from PyPDF2 import PdfReader

In [ ]:
# 전처리 함수
def clean_pdf_text(text: str) -> str:
    # 1. 줄바꿈을 공백으로 변환
    text = text.replace('\n', ' ')

    # 2. 연속된 공백을 하나로 압축
    text = re.sub(r'\s+', ' ', text)

    # 3. 특수기호 제거 (필요 없는 표시문자)
    text = re.sub(r'[■◆●※▶▷▲→◇]', '', text)

    # 4. "<표1>", "- 7 -" 같은 불필요한 표현 제거
    text = re.sub(r'<.*?>', '', text)  # <표1> 같은 표기 제거
    text = re.sub(r'-\s*\d+\s*-', '', text)  # - 7 - 같은 페이지 번호 제거

    return text.strip()

# 청크화 시키는 함수
def split_text_into_chunks(text, chunk_size=500):
    return [text[i:i+chunk_size].strip() for i in range(0, len(text), chunk_size)]

# ✅ 파일명 정규화 함수
def normalize_filename(name):
    return unicodedata.normalize("NFC", name)

In [ ]:
# 크롤링으로 텍스트 추출된 구글 드라이브에 저장된 마크다운 파일을 벡터 DB에 저장하기
# 중요: 현재 소프트웨어 학과 collection에 저장중 = software_department
import unicodedata

# ✅ .md 파일 ↔ URL 매핑 (소프트웨어학과)
software_md_urls = {
    # "문서 파일명": "출처 URL"

    # 교수진 폴더
    "교수진": "https://cse.kongju.ac.kr/ZD1110/15508/subview.do",

    # 규정자료실 폴더
    "K-Value up 미래설계 1,2 과목 이수기준 안내": "https://sw.kongju.ac.kr/ZD1180/11659/subview.do?enc=Zm5jdDF8QEB8JTJGYmJzJTJGWkQxMTgwJTJGMTQyNiUyRjI1MTM4MiUyRmFydGNsVmlldy5kbyUzRnBhZ2UlM0QxJTI2c3JjaENvbHVtbiUzRCUyNnNyY2hXcmQlM0QlMjZiYnNDbFNlcSUzRCUyNmJic09wZW5XcmRTZXElM0QlMjZyZ3NCZ25kZVN0ciUzRCUyNnJnc0VuZGRlU3RyJTNEJTI2aXNWaWV3TWluZSUzRGZhbHNlJTI2cGFzc3dvcmQlM0QlMjY%3D",
    "[소전] 졸업논문규정(20.04.01 개정)": "https://sw.kongju.ac.kr/ZD1180/11659/subview.do?enc=Zm5jdDF8QEB8JTJGYmJzJTJGWkQxMTgwJTJGMTQyNiUyRjIwODk2MCUyRmFydGNsVmlldy5kbyUzRnBhZ2UlM0QxJTI2c3JjaENvbHVtbiUzRCUyNnNyY2hXcmQlM0QlMjZiYnNDbFNlcSUzRCUyNmJic09wZW5XcmRTZXElM0QlMjZyZ3NCZ25kZVN0ciUzRCUyNnJnc0VuZGRlU3RyJTNEJTI2aXNWaWV3TWluZSUzRGZhbHNlJTI2cGFzc3dvcmQlM0QlMjY%3D",
    "[소전] 장학금규정(20.04.01개정)": "https://sw.kongju.ac.kr/ZD1180/11659/subview.do?enc=Zm5jdDF8QEB8JTJGYmJzJTJGWkQxMTgwJTJGMTQyNiUyRjIwODk1OSUyRmFydGNsVmlldy5kbyUzRnBhZ2UlM0QxJTI2c3JjaENvbHVtbiUzRCUyNnNyY2hXcmQlM0QlMjZiYnNDbFNlcSUzRCUyNmJic09wZW5XcmRTZXElM0QlMjZyZ3NCZ25kZVN0ciUzRCUyNnJnc0VuZGRlU3RyJTNEJTI2aXNWaWV3TWluZSUzRGZhbHNlJTI2cGFzc3dvcmQlM0QlMjY%3D",
    "[소프트웨어전공] 미래설계 교과목 이수기준 변경 안내": "https://sw.kongju.ac.kr/ZD1180/11659/subview.do?enc=Zm5jdDF8QEB8JTJGYmJzJTJGWkQxMTgwJTJGMTQyNiUyRjIwODk2NyUyRmFydGNsVmlldy5kbyUzRnBhZ2UlM0QxJTI2c3JjaENvbHVtbiUzRCUyNnNyY2hXcmQlM0QlMjZiYnNDbFNlcSUzRCUyNmJic09wZW5XcmRTZXElM0QlMjZyZ3NCZ25kZVN0ciUzRCUyNnJnc0VuZGRlU3RyJTNEJTI2aXNWaWV3TWluZSUzRGZhbHNlJTI2cGFzc3dvcmQlM0QlMjY%3D",
    "공결 규정입니다.": "https://sw.kongju.ac.kr/ZD1180/11659/subview.do?enc=Zm5jdDF8QEB8JTJGYmJzJTJGWkQxMTgwJTJGMTQyNiUyRjI5Mzg1OSUyRmFydGNsVmlldy5kbyUzRnBhZ2UlM0QxJTI2c3JjaENvbHVtbiUzRCUyNnNyY2hXcmQlM0QlMjZiYnNDbFNlcSUzRCUyNmJic09wZW5XcmRTZXElM0QlMjZyZ3NCZ25kZVN0ciUzRCUyNnJnc0VuZGRlU3RyJTNEJTI2aXNWaWV3TWluZSUzRGZhbHNlJTI2cGFzc3dvcmQlM0QlMjY%3D",

    # 대학원 폴더
    "대학원": "https://cse.kongju.ac.kr/ZD1110/15507/subview.do",

    # 비전 폴더
    "비전": "https://sw.kongju.ac.kr/ZD1180/11628/subview.do",

    # 서식자료실 폴더
    "공결승인신청서 서식": "https://sw.kongju.ac.kr/ZD1180/11660/subview.do?enc=Zm5jdDF8QEB8JTJGYmJzJTJGWkQxMTgwJTJGMTQyNyUyRjIwODk3MCUyRmFydGNsVmlldy5kbyUzRnBhZ2UlM0QxJTI2c3JjaENvbHVtbiUzRCUyNnNyY2hXcmQlM0QlMjZiYnNDbFNlcSUzRCUyNmJic09wZW5XcmRTZXElM0QlMjZyZ3NCZ25kZVN0ciUzRCUyNnJnc0VuZGRlU3RyJTNEJTI2aXNWaWV3TWluZSUzRGZhbHNlJTI2cGFzc3dvcmQlM0QlMjY%3D",
    "보안카드 발급신청서": "https://sw.kongju.ac.kr/ZD1180/11660/subview.do?enc=Zm5jdDF8QEB8JTJGYmJzJTJGWkQxMTgwJTJGMTQyNyUyRjI1MTI4MSUyRmFydGNsVmlldy5kbyUzRnBhZ2UlM0QxJTI2c3JjaENvbHVtbiUzRCUyNnNyY2hXcmQlM0QlMjZiYnNDbFNlcSUzRCUyNmJic09wZW5XcmRTZXElM0QlMjZyZ3NCZ25kZVN0ciUzRCUyNnJnc0VuZGRlU3RyJTNEJTI2aXNWaWV3TWluZSUzRGZhbHNlJTI2cGFzc3dvcmQlM0QlMjY%3D",
    "수강신청 포기 요청서": "https://sw.kongju.ac.kr/ZD1180/11660/subview.do?enc=Zm5jdDF8QEB8JTJGYmJzJTJGWkQxMTgwJTJGMTQyNyUyRjI1OTQ0MiUyRmFydGNsVmlldy5kbyUzRnBhZ2UlM0QxJTI2c3JjaENvbHVtbiUzRCUyNnNyY2hXcmQlM0QlMjZiYnNDbFNlcSUzRCUyNmJic09wZW5XcmRTZXElM0QlMjZyZ3NCZ25kZVN0ciUzRCUyNnJnc0VuZGRlU3RyJTNEJTI2aXNWaWV3TWluZSUzRGZhbHNlJTI2cGFzc3dvcmQlM0QlMjY%3D",
    "수강정원 초과 교과목 추가 수강 확인서": "https://sw.kongju.ac.kr/ZD1180/11660/subview.do?enc=Zm5jdDF8QEB8JTJGYmJzJTJGWkQxMTgwJTJGMTQyNyUyRjIwODk3MSUyRmFydGNsVmlldy5kbyUzRnBhZ2UlM0QxJTI2c3JjaENvbHVtbiUzRCUyNnNyY2hXcmQlM0QlMjZiYnNDbFNlcSUzRCUyNmJic09wZW5XcmRTZXElM0QlMjZyZ3NCZ25kZVN0ciUzRCUyNnJnc0VuZGRlU3RyJTNEJTI2aXNWaWV3TWluZSUzRGZhbHNlJTI2cGFzc3dvcmQlM0QlMjY%3D",
    "조기취업승인신청서 서식": "https://sw.kongju.ac.kr/ZD1180/11660/subview.do?enc=Zm5jdDF8QEB8JTJGYmJzJTJGWkQxMTgwJTJGMTQyNyUyRjIwODk2OSUyRmFydGNsVmlldy5kbyUzRnBhZ2UlM0QxJTI2c3JjaENvbHVtbiUzRCUyNnNyY2hXcmQlM0QlMjZiYnNDbFNlcSUzRCUyNmJic09wZW5XcmRTZXElM0QlMjZyZ3NCZ25kZVN0ciUzRCUyNnJnc0VuZGRlU3RyJTNEJTI2aXNWaWV3TWluZSUzRGZhbHNlJTI2cGFzc3dvcmQlM0QlMjY%3D",
    "졸업학점 미달자 추가 수강신청 요청서": "https://sw.kongju.ac.kr/ZD1180/11660/subview.do?enc=Zm5jdDF8QEB8JTJGYmJzJTJGWkQxMTgwJTJGMTQyNyUyRjI1OTQ0MSUyRmFydGNsVmlldy5kbyUzRnBhZ2UlM0QxJTI2c3JjaENvbHVtbiUzRCUyNnNyY2hXcmQlM0QlMjZiYnNDbFNlcSUzRCUyNmJic09wZW5XcmRTZXElM0QlMjZyZ3NCZ25kZVN0ciUzRCUyNnJnc0VuZGRlU3RyJTNEJTI2aXNWaWV3TWluZSUzRGZhbHNlJTI2cGFzc3dvcmQlM0QlMjY%3D",
    "컴퓨터공학전공 졸업논문 계획서 서식": "https://sw.kongju.ac.kr/ZD1180/11660/subview.do?enc=Zm5jdDF8QEB8JTJGYmJzJTJGWkQxMTgwJTJGMTQyNyUyRjIwODk2OCUyRmFydGNsVmlldy5kbyUzRnBhZ2UlM0QxJTI2c3JjaENvbHVtbiUzRCUyNnNyY2hXcmQlM0QlMjZiYnNDbFNlcSUzRCUyNmJic09wZW5XcmRTZXElM0QlMjZyZ3NCZ25kZVN0ciUzRCUyNnJnc0VuZGRlU3RyJTNEJTI2aXNWaWV3TWluZSUzRGZhbHNlJTI2cGFzc3dvcmQlM0QlMjY%3D",
    "폐강과목 추가수강요청서": "https://sw.kongju.ac.kr/ZD1180/11660/subview.do?enc=Zm5jdDF8QEB8JTJGYmJzJTJGWkQxMTgwJTJGMTQyNyUyRjI1OTUxMyUyRmFydGNsVmlldy5kbyUzRnBhZ2UlM0QxJTI2c3JjaENvbHVtbiUzRCUyNnNyY2hXcmQlM0QlMjZiYnNDbFNlcSUzRCUyNmJic09wZW5XcmRTZXElM0QlMjZyZ3NCZ25kZVN0ciUzRCUyNnJnc0VuZGRlU3RyJTNEJTI2aXNWaWV3TWluZSUzRGZhbHNlJTI2cGFzc3dvcmQlM0QlMjY%3D",

    # 연구실
    "연구실": "https://cse.kongju.ac.kr/ZD1110/11542/subview.do",

    # 연구실_공지사항
    "2021학년도 후기(2022.8월졸업) 일반대학원 석박사 학위청구논문 심사 계획 안내": "https://cse.kongju.ac.kr/ZD1110/11543/subview.do?enc=Zm5jdDF8QEB8JTJGYmJzJTJGWkQxMTEwJTJGMTkxNSUyRjI1MjU3MyUyRmFydGNsVmlldy5kbyUzRnBhZ2UlM0QxJTI2c3JjaENvbHVtbiUzRCUyNnNyY2hXcmQlM0QlMjZiYnNDbFNlcSUzRCUyNmJic09wZW5XcmRTZXElM0QlMjZyZ3NCZ25kZVN0ciUzRCUyNnJnc0VuZGRlU3RyJTNEJTI2aXNWaWV3TWluZSUzRGZhbHNlJTI2cGFzc3dvcmQlM0QlMjY%3D",
    "2022학년도 전기 대학원 석？박사 학위청구논문 심사계획 안내": "https://cse.kongju.ac.kr/ZD1110/11543/subview.do?enc=Zm5jdDF8QEB8JTJGYmJzJTJGWkQxMTEwJTJGMTkxNSUyRjI2MDU0MCUyRmFydGNsVmlldy5kbyUzRnBhZ2UlM0QxJTI2c3JjaENvbHVtbiUzRCUyNnNyY2hXcmQlM0QlMjZiYnNDbFNlcSUzRCUyNmJic09wZW5XcmRTZXElM0QlMjZyZ3NCZ25kZVN0ciUzRCUyNnJnc0VuZGRlU3RyJTNEJTI2aXNWaWV3TWluZSUzRGZhbHNlJTI2cGFzc3dvcmQlM0QlMjY%3D",
    "2022학년도 후기(2023.8월 졸업) 석？박사 학위 청구 논문 심사계획": "https://cse.kongju.ac.kr/ZD1110/11543/subview.do?enc=Zm5jdDF8QEB8JTJGYmJzJTJGWkQxMTEwJTJGMTkxNSUyRjI3MTczNCUyRmFydGNsVmlldy5kbyUzRnBhZ2UlM0QxJTI2c3JjaENvbHVtbiUzRCUyNnNyY2hXcmQlM0QlMjZiYnNDbFNlcSUzRCUyNmJic09wZW5XcmRTZXElM0QlMjZyZ3NCZ25kZVN0ciUzRCUyNnJnc0VuZGRlU3RyJTNEJTI2aXNWaWV3TWluZSUzRGZhbHNlJTI2cGFzc3dvcmQlM0QlMjY%3D",
    "2023학년도 1학기 대학원생 우수학술지 논문게재 특별장학금 지급 계획 안내": "https://cse.kongju.ac.kr/ZD1110/11543/subview.do?enc=Zm5jdDF8QEB8JTJGYmJzJTJGWkQxMTEwJTJGMTkxNSUyRjI3Mzg3MSUyRmFydGNsVmlldy5kbyUzRnBhZ2UlM0QxJTI2c3JjaENvbHVtbiUzRCUyNnNyY2hXcmQlM0QlMjZiYnNDbFNlcSUzRCUyNmJic09wZW5XcmRTZXElM0QlMjZyZ3NCZ25kZVN0ciUzRCUyNnJnc0VuZGRlU3RyJTNEJTI2aXNWaWV3TWluZSUzRGZhbHNlJTI2cGFzc3dvcmQlM0QlMjY%3D",
    "2023학년도 전기 석？박사 학위 청구 논문 심사계획 안내": "https://cse.kongju.ac.kr/ZD1110/11543/subview.do?enc=Zm5jdDF8QEB8JTJGYmJzJTJGWkQxMTEwJTJGMTkxNSUyRjI3ODY5MyUyRmFydGNsVmlldy5kbyUzRnBhZ2UlM0QxJTI2c3JjaENvbHVtbiUzRCUyNnNyY2hXcmQlM0QlMjZiYnNDbFNlcSUzRCUyNmJic09wZW5XcmRTZXElM0QlMjZyZ3NCZ25kZVN0ciUzRCUyNnJnc0VuZGRlU3RyJTNEJTI2aXNWaWV3TWluZSUzRGZhbHNlJTI2cGFzc3dvcmQlM0QlMjY%3D",
    "2024학년도 2학기 일반대학원 논문제출 자격시험(외국어 및 종합시험) 시행계획 안내": "https://cse.kongju.ac.kr/ZD1110/11543/subview.do?enc=Zm5jdDF8QEB8JTJGYmJzJTJGWkQxMTEwJTJGMTkxNSUyRjI5OTg0NiUyRmFydGNsVmlldy5kbyUzRnBhZ2UlM0QxJTI2c3JjaENvbHVtbiUzRCUyNnNyY2hXcmQlM0QlMjZiYnNDbFNlcSUzRCUyNmJic09wZW5XcmRTZXElM0QlMjZyZ3NCZ25kZVN0ciUzRCUyNnJnc0VuZGRlU3RyJTNEJTI2aXNWaWV3TWluZSUzRGZhbHNlJTI2cGFzc3dvcmQlM0QlMjY%3D",
    "2024학년도 전기(2025년 2월 졸업) 일반대학원 석박사 학위청구논문 심사계획 안내": "https://cse.kongju.ac.kr/ZD1110/11543/subview.do?enc=Zm5jdDF8QEB8JTJGYmJzJTJGWkQxMTEwJTJGMTkxNSUyRjMwMDk3NCUyRmFydGNsVmlldy5kbyUzRnBhZ2UlM0QxJTI2c3JjaENvbHVtbiUzRCUyNnNyY2hXcmQlM0QlMjZiYnNDbFNlcSUzRCUyNmJic09wZW5XcmRTZXElM0QlMjZyZ3NCZ25kZVN0ciUzRCUyNnJnc0VuZGRlU3RyJTNEJTI2aXNWaWV3TWluZSUzRGZhbHNlJTI2cGFzc3dvcmQlM0QlMjY%3D",
    "2025학년도 1학기 일반대학원 논문제출 자격시험(외국어, 종합시험) 시행계획 안내": "https://cse.kongju.ac.kr/ZD1110/11543/subview.do?enc=Zm5jdDF8QEB8JTJGYmJzJTJGWkQxMTEwJTJGMTkxNSUyRjQwNjYxNSUyRmFydGNsVmlldy5kbyUzRnBhZ2UlM0QxJTI2c3JjaENvbHVtbiUzRCUyNnNyY2hXcmQlM0QlMjZiYnNDbFNlcSUzRCUyNmJic09wZW5XcmRTZXElM0QlMjZyZ3NCZ25kZVN0ciUzRCUyNnJnc0VuZGRlU3RyJTNEJTI2aXNWaWV3TWluZSUzRGZhbHNlJTI2cGFzc3dvcmQlM0QlMjY%3D",
    "2025학년도 전일제(RATA)장학생 선발 안내": "https://cse.kongju.ac.kr/ZD1110/11543/subview.do?enc=Zm5jdDF8QEB8JTJGYmJzJTJGWkQxMTEwJTJGMTkxNSUyRjQwNzAwMCUyRmFydGNsVmlldy5kbyUzRnBhZ2UlM0QxJTI2c3JjaENvbHVtbiUzRCUyNnNyY2hXcmQlM0QlMjZiYnNDbFNlcSUzRCUyNmJic09wZW5XcmRTZXElM0QlMjZyZ3NCZ25kZVN0ciUzRCUyNnJnc0VuZGRlU3RyJTNEJTI2aXNWaWV3TWluZSUzRGZhbHNlJTI2cGFzc3dvcmQlM0QlMjY%3D",
    "논문작성계획서 제출 요청(2학기생)": "https://cse.kongju.ac.kr/ZD1110/11543/subview.do?enc=Zm5jdDF8QEB8JTJGYmJzJTJGWkQxMTEwJTJGMTkxNSUyRjI1MjY5NiUyRmFydGNsVmlldy5kbyUzRnBhZ2UlM0QxJTI2c3JjaENvbHVtbiUzRCUyNnNyY2hXcmQlM0QlMjZiYnNDbFNlcSUzRCUyNmJic09wZW5XcmRTZXElM0QlMjZyZ3NCZ25kZVN0ciUzRCUyNnJnc0VuZGRlU3RyJTNEJTI2aXNWaWV3TWluZSUzRGZhbHNlJTI2cGFzc3dvcmQlM0QlMjY%3D",

    # 연혁
    "연혁": "https://sw.kongju.ac.kr/ZD1180/11622/subview.do",

    # 인사말
    "인사말": "https://sw.kongju.ac.kr/ZD1180/11621/subview.do",

    # 특강공지
    "2021년 4대 폭력 예방교육(학세직, K밸류업1 실적 인정)": "https://sw.kongju.ac.kr/ZD1180/11655/subview.do?enc=Zm5jdDF8QEB8JTJGYmJzJTJGWkQxMTgwJTJGMTQyNCUyRjIwODk1NiUyRmFydGNsVmlldy5kbyUzRnBhZ2UlM0QxJTI2c3JjaENvbHVtbiUzRCUyNnNyY2hXcmQlM0QlMjZiYnNDbFNlcSUzRCUyNmJic09wZW5XcmRTZXElM0QlMjZyZ3NCZ25kZVN0ciUzRCUyNnJnc0VuZGRlU3RyJTNEJTI2aXNWaWV3TWluZSUzRGZhbHNlJTI2cGFzc3dvcmQlM0QlMjY%3D",
    "2021년 글로벌 역량강화를 위한 정보탐색 프로그램(학세직, K밸류업1 실적 인정)": "https://sw.kongju.ac.kr/ZD1180/11655/subview.do?enc=Zm5jdDF8QEB8JTJGYmJzJTJGWkQxMTgwJTJGMTQyNCUyRjIwODk1NSUyRmFydGNsVmlldy5kbyUzRnBhZ2UlM0QxJTI2c3JjaENvbHVtbiUzRCUyNnNyY2hXcmQlM0QlMjZiYnNDbFNlcSUzRCUyNmJic09wZW5XcmRTZXElM0QlMjZyZ3NCZ25kZVN0ciUzRCUyNnJnc0VuZGRlU3RyJTNEJTI2aXNWaWV3TWluZSUzRGZhbHNlJTI2cGFzc3dvcmQlM0QlMjY%3D",
    "2021학년도 1학기 모두의 학교-학습법 워크숍(학세직, K밸류업1 실적 인정)": "https://sw.kongju.ac.kr/ZD1180/11655/subview.do?enc=Zm5jdDF8QEB8JTJGYmJzJTJGWkQxMTgwJTJGMTQyNCUyRjIwODk1MCUyRmFydGNsVmlldy5kbyUzRnBhZ2UlM0QxJTI2c3JjaENvbHVtbiUzRCUyNnNyY2hXcmQlM0QlMjZiYnNDbFNlcSUzRCUyNmJic09wZW5XcmRTZXElM0QlMjZyZ3NCZ25kZVN0ciUzRCUyNnJnc0VuZGRlU3RyJTNEJTI2aXNWaWV3TWluZSUzRGZhbHNlJTI2cGFzc3dvcmQlM0QlMjY%3D",
    "2021학년도 1학기 자기주도 학습역량 강화를 위한 학습법 워크숍 「모두의 학교」(학세직, K밸류업1 실적 인정)": "https://sw.kongju.ac.kr/ZD1180/11655/subview.do?enc=Zm5jdDF8QEB8JTJGYmJzJTJGWkQxMTgwJTJGMTQyNCUyRjIwODk0OSUyRmFydGNsVmlldy5kbyUzRnBhZ2UlM0QxJTI2c3JjaENvbHVtbiUzRCUyNnNyY2hXcmQlM0QlMjZiYnNDbFNlcSUzRCUyNmJic09wZW5XcmRTZXElM0QlMjZyZ3NCZ25kZVN0ciUzRCUyNnJnc0VuZGRlU3RyJTNEJTI2aXNWaWV3TWluZSUzRGZhbHNlJTI2cGFzc3dvcmQlM0QlMjY%3D",
    "2021학년도 1학기 중견기업 바로알기 특강(학세직, K밸류업1 실적 인정)": "https://sw.kongju.ac.kr/ZD1180/11655/subview.do?enc=Zm5jdDF8QEB8JTJGYmJzJTJGWkQxMTgwJTJGMTQyNCUyRjIwODk1MyUyRmFydGNsVmlldy5kbyUzRnBhZ2UlM0QxJTI2c3JjaENvbHVtbiUzRCUyNnNyY2hXcmQlM0QlMjZiYnNDbFNlcSUzRCUyNmJic09wZW5XcmRTZXElM0QlMjZyZ3NCZ25kZVN0ciUzRCUyNnJnc0VuZGRlU3RyJTNEJTI2aXNWaWV3TWluZSUzRGZhbHNlJTI2cGFzc3dvcmQlM0QlMjY%3D",
    "2021학년도 1학기 찾아가는 창업특강(10회차~11회차) 개최(학세직, K밸류업1 실적 인정)": "https://sw.kongju.ac.kr/ZD1180/11655/subview.do?enc=Zm5jdDF8QEB8JTJGYmJzJTJGWkQxMTgwJTJGMTQyNCUyRjIwODk1NyUyRmFydGNsVmlldy5kbyUzRnBhZ2UlM0QxJTI2c3JjaENvbHVtbiUzRCUyNnNyY2hXcmQlM0QlMjZiYnNDbFNlcSUzRCUyNmJic09wZW5XcmRTZXElM0QlMjZyZ3NCZ25kZVN0ciUzRCUyNnJnc0VuZGRlU3RyJTNEJTI2aXNWaWV3TWluZSUzRGZhbHNlJTI2cGFzc3dvcmQlM0QlMjY%3D",
    "2021학년도 1학기 채용설명회(학세직, K밸류업1 실적 인정)": "https://sw.kongju.ac.kr/ZD1180/11655/subview.do?enc=Zm5jdDF8QEB8JTJGYmJzJTJGWkQxMTgwJTJGMTQyNCUyRjIwODk1NCUyRmFydGNsVmlldy5kbyUzRnBhZ2UlM0QxJTI2c3JjaENvbHVtbiUzRCUyNnNyY2hXcmQlM0QlMjZiYnNDbFNlcSUzRCUyNmJic09wZW5XcmRTZXElM0QlMjZyZ3NCZ25kZVN0ciUzRCUyNnJnc0VuZGRlU3RyJTNEJTI2aXNWaWV3TWluZSUzRGZhbHNlJTI2cGFzc3dvcmQlM0QlMjY%3D",
    "2021학년도 공공기관 취업 전략 설명회 운영(학세직, K밸류업1 실적 인정)": "https://sw.kongju.ac.kr/ZD1180/11655/subview.do?enc=Zm5jdDF8QEB8JTJGYmJzJTJGWkQxMTgwJTJGMTQyNCUyRjIwODk1MiUyRmFydGNsVmlldy5kbyUzRnBhZ2UlM0QxJTI2c3JjaENvbHVtbiUzRCUyNnNyY2hXcmQlM0QlMjZiYnNDbFNlcSUzRCUyNmJic09wZW5XcmRTZXElM0QlMjZyZ3NCZ25kZVN0ciUzRCUyNnJnc0VuZGRlU3RyJTNEJTI2aXNWaWV3TWluZSUzRGZhbHNlJTI2cGFzc3dvcmQlM0QlMjY%3D",
    "2021학년도 심리검사 활용 집단상담 프로그램 신청 안내(학세직, K밸류업1 실적 인정)": "https://sw.kongju.ac.kr/ZD1180/11655/subview.do?enc=Zm5jdDF8QEB8JTJGYmJzJTJGWkQxMTgwJTJGMTQyNCUyRjIwODk1OCUyRmFydGNsVmlldy5kbyUzRnBhZ2UlM0QxJTI2c3JjaENvbHVtbiUzRCUyNnNyY2hXcmQlM0QlMjZiYnNDbFNlcSUzRCUyNmJic09wZW5XcmRTZXElM0QlMjZyZ3NCZ25kZVN0ciUzRCUyNnJnc0VuZGRlU3RyJTNEJTI2aXNWaWV3TWluZSUzRGZhbHNlJTI2cGFzc3dvcmQlM0QlMjY%3D",
    "2021학년도 지감인(智感仁) 운영(학세직, K밸류업1 실적인정)": "https://sw.kongju.ac.kr/ZD1180/11655/subview.do?enc=Zm5jdDF8QEB8JTJGYmJzJTJGWkQxMTgwJTJGMTQyNCUyRjIwODk1MSUyRmFydGNsVmlldy5kbyUzRnBhZ2UlM0QxJTI2c3JjaENvbHVtbiUzRCUyNnNyY2hXcmQlM0QlMjZiYnNDbFNlcSUzRCUyNmJic09wZW5XcmRTZXElM0QlMjZyZ3NCZ25kZVN0ciUzRCUyNnJnc0VuZGRlU3RyJTNEJTI2aXNWaWV3TWluZSUzRGZhbHNlJTI2cGFzc3dvcmQlM0QlMjY%3D",

    # 학과 공지
    "2024학년도 동계 계절학기 미래설계(전필) 1~6 증빙 파일 제출 안내": "https://sw.kongju.ac.kr/ZD1180/11654/subview.do?enc=Zm5jdDF8QEB8JTJGYmJzJTJGWkQxMTgwJTJGMTQyMyUyRjQwMDU3NiUyRmFydGNsVmlldy5kbyUzRnBhZ2UlM0QxJTI2c3JjaENvbHVtbiUzRCUyNnNyY2hXcmQlM0QlMjZiYnNDbFNlcSUzRCUyNmJic09wZW5XcmRTZXElM0QlMjZyZ3NCZ25kZVN0ciUzRCUyNnJnc0VuZGRlU3RyJTNEJTI2aXNWaWV3TWluZSUzRGZhbHNlJTI2cGFzc3dvcmQlM0QlMjY%3D",
    "2024학년도 취업역량강화 장학생 선발 안내": "https://sw.kongju.ac.kr/ZD1180/11654/subview.do?enc=Zm5jdDF8QEB8JTJGYmJzJTJGWkQxMTgwJTJGMTQyMyUyRjMwMTQ5NiUyRmFydGNsVmlldy5kbyUzRnBhZ2UlM0QxJTI2c3JjaENvbHVtbiUzRCUyNnNyY2hXcmQlM0QlMjZiYnNDbFNlcSUzRCUyNmJic09wZW5XcmRTZXElM0QlMjZyZ3NCZ25kZVN0ciUzRCUyNnJnc0VuZGRlU3RyJTNEJTI2aXNWaWV3TWluZSUzRGZhbHNlJTI2cGFzc3dvcmQlM0QlMjY%3D",
    "2025학년도 1학기 성적우수장학금 증빙 확인 바람_0212": "https://sw.kongju.ac.kr/ZD1180/11654/subview.do?enc=Zm5jdDF8QEB8JTJGYmJzJTJGWkQxMTgwJTJGMTQyMyUyRjQwMjE4NSUyRmFydGNsVmlldy5kbyUzRnBhZ2UlM0QxJTI2c3JjaENvbHVtbiUzRCUyNnNyY2hXcmQlM0QlMjZiYnNDbFNlcSUzRCUyNmJic09wZW5XcmRTZXElM0QlMjZyZ3NCZ25kZVN0ciUzRCUyNnJnc0VuZGRlU3RyJTNEJTI2aXNWaWV3TWluZSUzRGZhbHNlJTI2cGFzc3dvcmQlM0QlMjY%3D",
    "2025학년도 1학기 추가수강 안내": "https://sw.kongju.ac.kr/ZD1180/11654/subview.do?enc=Zm5jdDF8QEB8JTJGYmJzJTJGWkQxMTgwJTJGMTQyMyUyRjQwNTU2MCUyRmFydGNsVmlldy5kbyUzRnBhZ2UlM0QxJTI2c3JjaENvbHVtbiUzRCUyNnNyY2hXcmQlM0QlMjZiYnNDbFNlcSUzRCUyNmJic09wZW5XcmRTZXElM0QlMjZyZ3NCZ25kZVN0ciUzRCUyNnJnc0VuZGRlU3RyJTNEJTI2aXNWaWV3TWluZSUzRGZhbHNlJTI2cGFzc3dvcmQlM0QlMjY%3D",
    "2025학년도 신입생 OT자료_과목수정": "https://sw.kongju.ac.kr/ZD1180/11654/subview.do?enc=Zm5jdDF8QEB8JTJGYmJzJTJGWkQxMTgwJTJGMTQyMyUyRjQwNDczMyUyRmFydGNsVmlldy5kbyUzRnBhZ2UlM0QxJTI2c3JjaENvbHVtbiUzRCUyNnNyY2hXcmQlM0QlMjZiYnNDbFNlcSUzRCUyNmJic09wZW5XcmRTZXElM0QlMjZyZ3NCZ25kZVN0ciUzRCUyNnJnc0VuZGRlU3RyJTNEJTI2aXNWaWV3TWluZSUzRGZhbHNlJTI2cGFzc3dvcmQlM0QlMjY%3D",
    "K-Value up 미래설계 증빙파일 제출 안내": "https://sw.kongju.ac.kr/ZD1180/11654/subview.do?enc=Zm5jdDF8QEB8JTJGYmJzJTJGWkQxMTgwJTJGMTQyMyUyRjM5ODgxMSUyRmFydGNsVmlldy5kbyUzRnBhZ2UlM0QxJTI2c3JjaENvbHVtbiUzRCUyNnNyY2hXcmQlM0QlMjZiYnNDbFNlcSUzRCUyNmJic09wZW5XcmRTZXElM0QlMjZyZ3NCZ25kZVN0ciUzRCUyNnJnc0VuZGRlU3RyJTNEJTI2aXNWaWV3TWluZSUzRGZhbHNlJTI2cGFzc3dvcmQlM0QlMjY%3D",
    "SW중심대학사업 2025학년도 1학기 산학캡스톤디자인 모집공고": "https://sw.kongju.ac.kr/ZD1180/11654/subview.do?enc=Zm5jdDF8QEB8JTJGYmJzJTJGWkQxMTgwJTJGMTQyMyUyRjQwNjE5NyUyRmFydGNsVmlldy5kbyUzRnBhZ2UlM0QxJTI2c3JjaENvbHVtbiUzRCUyNnNyY2hXcmQlM0QlMjZiYnNDbFNlcSUzRCUyNmJic09wZW5XcmRTZXElM0QlMjZyZ3NCZ25kZVN0ciUzRCUyNnJnc0VuZGRlU3RyJTNEJTI2aXNWaWV3TWluZSUzRGZhbHNlJTI2cGFzc3dvcmQlM0QlMjY%3D",
    "[전학년] 2025학년도 1학기 수강신청 안내": "https://sw.kongju.ac.kr/ZD1180/11654/subview.do?enc=Zm5jdDF8QEB8JTJGYmJzJTJGWkQxMTgwJTJGMTQyMyUyRjQwMzIyNSUyRmFydGNsVmlldy5kbyUzRnBhZ2UlM0QxJTI2c3JjaENvbHVtbiUzRCUyNnNyY2hXcmQlM0QlMjZiYnNDbFNlcSUzRCUyNmJic09wZW5XcmRTZXElM0QlMjZyZ3NCZ25kZVN0ciUzRCUyNnJnc0VuZGRlU3RyJTNEJTI2aXNWaWV3TWluZSUzRGZhbHNlJTI2cGFzc3dvcmQlM0QlMjY%3D",
    "디지털기업-청년인재 간의 만남의 장 신청 안내": "https://sw.kongju.ac.kr/ZD1180/11654/subview.do?enc=Zm5jdDF8QEB8JTJGYmJzJTJGWkQxMTgwJTJGMTQyMyUyRjM5ODU1OSUyRmFydGNsVmlldy5kbyUzRnBhZ2UlM0QxJTI2c3JjaENvbHVtbiUzRCUyNnNyY2hXcmQlM0QlMjZiYnNDbFNlcSUzRCUyNmJic09wZW5XcmRTZXElM0QlMjZyZ3NCZ25kZVN0ciUzRCUyNnJnc0VuZGRlU3RyJTNEJTI2aXNWaWV3TWluZSUzRGZhbHNlJTI2cGFzc3dvcmQlM0QlMjY%3D",
    "자기주도적 역량강화 프로그램 장학금 신청 안내": "https://sw.kongju.ac.kr/ZD1180/11654/subview.do?enc=Zm5jdDF8QEB8JTJGYmJzJTJGWkQxMTgwJTJGMTQyMyUyRjQwMTgzOSUyRmFydGNsVmlldy5kbyUzRnBhZ2UlM0QxJTI2c3JjaENvbHVtbiUzRCUyNnNyY2hXcmQlM0QlMjZiYnNDbFNlcSUzRCUyNmJic09wZW5XcmRTZXElM0QlMjZyZ3NCZ25kZVN0ciUzRCUyNnJnc0VuZGRlU3RyJTNEJTI2aXNWaWV3TWluZSUzRGZhbHNlJTI2cGFzc3dvcmQlM0QlMjY%3D",
}

# ✅ URL 매핑을 위한 정규화 딕셔너리 생성
normalized_url_map = {
    unicodedata.normalize("NFC", k): v for k, v in software_md_urls.items()
}

# ✅ 경로 설정
md_folder_path = "/content/drive/MyDrive/html/software_department/학과공지"
persist_dir = "/content/drive/MyDrive/chroma_index"

# ✅ Chroma DB 연결
client = PersistentClient(path=persist_dir)
collection = client.get_or_create_collection(name="software_department")
existing_ids = set(collection.get(limit=None)["ids"])

# ✅ 임베딩 모델 로딩
embedding_model = SentenceTransformer("sentence-transformers/all-MiniLM-L12-v2")

# ✅ 마크다운 파일 처리
file_list = [f for f in os.listdir(md_folder_path) if f.endswith(".md")]
total_added = 0

for filename in file_list:
    md_path = os.path.join(md_folder_path, filename)
    base_name = os.path.splitext(filename)[0]
    normalized_name = unicodedata.normalize("NFC", base_name)

    # ✅ 텍스트 불러오기
    with open(md_path, "r", encoding="utf-8") as f:
        raw_text = f.read()

    if not raw_text.strip():
        print(f"⚠️ {filename}: 텍스트 없음 → 스킵")
        continue

    # ✅ 전처리 + 청크화 + 임베딩
    cleaned_text = clean_pdf_text(raw_text)
    chunks = split_text_into_chunks(cleaned_text)
    embeddings = embedding_model.encode(chunks, show_progress_bar=True).tolist()
    ids = [f"{normalized_name}_chunk_{i}" for i in range(len(chunks))]

    # ✅ 중복 ID 필터링
    new_chunks, new_embeddings, new_ids, new_metadatas = [], [], [], []
    for chunk, emb, id_ in zip(chunks, embeddings, ids):
        if id_ not in existing_ids:
            new_chunks.append(chunk)
            new_embeddings.append(emb)
            new_ids.append(id_)
            source_url = normalized_url_map.get(normalized_name)
            if not source_url:
                print(f"출처 URL 미등록: {normalized_name}")

            new_metadatas.append({
                "date": str(date.today()),
                "department": "공주대학교 소프트웨어학과",
                "type": "markdown",
                "source_url": source_url or "출처 URL 미등록"
            })

    # ✅ Chroma DB에 저장
    if new_chunks:
        collection.add(
            documents=new_chunks,
            embeddings=new_embeddings,
            metadatas=new_metadatas,
            ids=new_ids
        )
        print(f"✅ {filename}: {len(new_chunks)}개 저장 완료")
        total_added += len(new_chunks)
    else:
        print(f"⚠️ {filename}: 모든 청크가 이미 존재함 → 스킵")

print(f"\n🎉 최종 저장된 청크 수: {total_added}")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

⚠️ K-Value up 미래설계 증빙파일 제출 안내.md: 모든 청크가 이미 존재함 → 스킵


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

⚠️ 디지털기업-청년인재 간의 만남의 장 신청 안내.md: 모든 청크가 이미 존재함 → 스킵


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

⚠️ 2024학년도 취업역량강화 장학생 선발 안내.md: 모든 청크가 이미 존재함 → 스킵

🎉 최종 저장된 청크 수: 0


In [ ]:
# 구글 드라이브에 존재하는 pdf 파일 벡터DB에 저장하기
# 중요: 현재 소프트웨어 학과 collection에 저장중 = software_department
import unicodedata

# ✅ PDF ↔ URL 매핑 사전(소프트웨어 학과)
software_pdf_urls = {    # 추가 변경
    "소프트웨어전공 졸업규정": "https://sw.kongju.ac.kr/synap/skin/doc.html?fn=ZD1180_1426_208960_3&rs=/synap/result/bbs/1426",
    "소프트웨어전공 장학규정": "https://sw.kongju.ac.kr/synap/skin/doc.html?fn=ZD1180_1426_208959_2&rs=/synap/result/bbs/1426",
    "미래설계 교과목 이수기준(2021.04.19.)" : "https://sw.kongju.ac.kr/synap/skin/doc.html?fn=ZD1180_1426_208967_1&rs=/synap/result/bbs/1426",
    # 👇 여기에 새로운 문서를 추가해 주세요
    # "문서파일명": "출처 URL"
}

# ✅ 경로 설정
pdf_folder_path = "/content/drive/MyDrive/pdf/software_department"      # collection 변경(변경이란 단어 없으면 코드 절대 바꾸지 마삼)
persist_dir = "/content/drive/MyDrive/chroma_index"

# ✅ Chroma DB 연결
client = PersistentClient(path=persist_dir)
collection = client.get_or_create_collection(name="software_department")    # name 변경
existing_ids = set(collection.get(limit=None)["ids"])

# ✅ 임베딩 모델 로딩
embedding_model = SentenceTransformer("sentence-transformers/all-MiniLM-L12-v2")

# ✅ PDF 폴더 내 전체 파일 처리
file_list = [f for f in os.listdir(pdf_folder_path) if f.endswith(".pdf")]
total_added = 0

for filename in file_list:
    pdf_path = os.path.join(pdf_folder_path, filename)
    base_name = os.path.splitext(filename)[0]

    # 🔧 파일명을 NFC 형태로 정규화
    normalized_name = unicodedata.normalize("NFC", base_name)

    # ✅ PDF 텍스트 추출
    reader = PdfReader(pdf_path)
    raw_text = "".join([page.extract_text() for page in reader.pages if page.extract_text()])
    if not raw_text.strip():
        print(f"⚠️ {filename}: 텍스트 없음 → 스킵")
        continue

    # ✅ 전처리 + 청크화 + 임베딩
    cleaned_text = clean_pdf_text(raw_text)
    chunks = split_text_into_chunks(cleaned_text)
    embeddings = embedding_model.encode(chunks, show_progress_bar=True).tolist()
    ids = [f"{normalized_name}_chunk_{i}" for i in range(len(chunks))]

    # ✅ 중복 ID 필터링
    new_chunks, new_embeddings, new_ids, new_metadatas = [], [], [], []
    for chunk, emb, id_ in zip(chunks, embeddings, ids):
        if id_ not in existing_ids:
            new_chunks.append(chunk)
            new_embeddings.append(emb)
            new_ids.append(id_)
            new_metadatas.append({
                "date": str(date.today()),
                "department": "공주대학교 소프트웨어학과",
                "type": "pdf",
                "source_url": software_pdf_urls.get(normalized_name, "출처 URL 미등록")
            })

    # ✅ Chroma DB에 저장
    if new_chunks:
        collection.add(
            documents=new_chunks,
            embeddings=new_embeddings,
            metadatas=new_metadatas,
            ids=new_ids
        )
        print(f"✅ {filename}: {len(new_chunks)}개 저장 완료")
        total_added += len(new_chunks)
    else:
        print(f"⚠️ {filename}: 모든 청크가 이미 존재함 → 스킵")

print(f"\n🎉 최종 저장된 청크 수: {total_added}")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

✅ 소프트웨어전공 졸업규정.pdf: 4개 저장 완료


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

✅ 소프트웨어전공 장학규정.pdf: 2개 저장 완료


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

✅ 미래설계 교과목 이수기준(2021.04.19.).pdf: 2개 저장 완료

🎉 최종 저장된 청크 수: 8


In [ ]:
# 저장된 결과 보기
# 1. 필요한 모듈 임포트
from chromadb import PersistentClient

# 2. ChromaDB 클라이언트 연결
persist_dir = "/content/drive/MyDrive/chroma_index"  # 실제 저장된 경로로 설정
client = PersistentClient(path=persist_dir)

# 저장된 컬렉션 불러오기 (컬렉션 이름은 실제 생성한 이름으로!)
collection = client.get_or_create_collection(name="software_department")

# 전체 문서 + 메타데이터 + 임베딩 가져오기
results = collection.get(include=["documents", "metadatas", "embeddings"])

# 결과 출력
print(f"✅ 저장된 총 청크 수: {len(results['documents'])}")
for i in range(len(results['documents'])):
    print(f"🆔 ID: {results['ids'][i]}")
    print(f"📄 청크 내용: {results['documents'][i][:200]}...")  # 너무 길면 앞부분만
    print(f"📎 메타데이터: {results['metadatas'][i]}")
    print(f"🔢 임베딩 벡터 (일부): {results['embeddings'][i][:5]}...")  # 임베딩 앞부분 5개만 보기
    print("-" * 80)


✅ 저장된 총 청크 수: 108
🆔 ID: 소프트웨어전공 졸업규정_chunk_0
📄 청크 내용: 소프트웨어전공 졸업논문 규정 제1조(목적) 이 내규는 공주대학교 학칙(이하 “학칙”이라 함) 제94조 및 공주대학교 학사운영 규정(이하 “학사규정 ”이라 함) 제12장에 의거 소프트웨어전공 , 컴퓨터소프트웨어공학전공 및 멀티미디어공학전공의 졸업논문에 관한 세부사항을 규정함을 목적으로 한다. 제2조(졸업논문 제출자격 ) ① 최종학기 등록을 필한 자라야 한다....
📎 메타데이터: {'date': '2025-03-27', 'department': '공주대학교 소프트웨어학과', 'source_url': 'https://sw.kongju.ac.kr/synap/skin/doc.html?fn=ZD1180_1426_208960_3&rs=/synap/result/bbs/1426', 'type': 'pdf'}
🔢 임베딩 벡터 (일부): [ 0.06352967  0.0725045   0.05380703  0.05122475 -0.06273629]...
--------------------------------------------------------------------------------
🆔 ID: 소프트웨어전공 졸업규정_chunk_1
📄 청크 내용: 지 발표하고 , 종강전까지 인쇄본을 제출하여야한다 ② 단독연구만 을 인정한다 . ③ 지도교수는 졸업논문의 자료준비 , 작성 및 발표 등에 관한 사항을 지도한다 . ④ 졸업논문의 제출 부수는 3부(심사위원 2부, 학과사무실 1부) 제출한다 제6조(졸업논문 심사위원 ) ① 졸업논문 심사위원은 지도교수를 포함한 2인 이상으로 하며 지도 교수의 추천에 의하여 소속...
📎 메타데이터: {'date': '2025-03-27', 'department': '공주대학교 소프트웨어학과', 'source_url': 'https://sw.kongju.ac.kr/synap/skin/doc.html?fn=ZD1180_1426_208960_3&rs=/sy

In [ ]:
# URL 안들어간거 삭제하기 *****************************************사용 조심********************************************
from chromadb import PersistentClient

# 1. ChromaDB 클라이언트 연결
persist_dir = "/content/drive/MyDrive/chroma_index"
client = PersistentClient(path=persist_dir)

# 2. 컬렉션 로드
collection = client.get_or_create_collection(name="software_department")

# 3. 전체 문서 가져오기 (ids는 자동 포함됨)
all_docs = collection.get(include=["metadatas"])

# 4. 삭제 대상 ID 추출
ids_to_delete = [
    doc_id
    for doc_id, metadata in zip(all_docs["ids"], all_docs["metadatas"])
    if metadata.get("source_url") == "출처 URL 미등록"
]

# 5. 삭제 수행
if ids_to_delete:
    print(f"삭제할 문서 수: {len(ids_to_delete)}개")
    collection.delete(ids=ids_to_delete)
else:
    print("삭제할 문서가 없습니다.")


삭제할 문서가 없습니다.


In [ ]:
!pip install bitsandbytes --upgrade --quiet

# ✅ Mistral 모델 로딩 (L4 GPU 대응: 8bit 양자화 + 로컬)
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch

model_path = "/content/drive/MyDrive/mistral_model"
tokenizer = AutoTokenizer.from_pretrained(model_path, local_files_only=True)

bnb_config = BitsAndBytesConfig(
    load_in_8bit=True,
    llm_int8_threshold=6.0
)

model = AutoModelForCausalLM.from_pretrained(
    "/content/drive/MyDrive/mistral_model",
    device_map="auto",
    torch_dtype=torch.float16,
    local_files_only=True
)

print("✅ 모델 로딩 완료")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 MB 30.6 MB/s eta 0:00:00


Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

✅ 모델 로딩 완료


In [ ]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from chromadb import PersistentClient
import torch

# ✅ 사용자 질문
query = "소프트웨어전공 장학생 선발 규정을 알려줘 pdf로 확인해줘"

# ✅ DB 연결
persist_dir = "/content/drive/MyDrive/chroma_index"
client = PersistentClient(path=persist_dir)
collection = client.get_collection(name="software_department")
results = collection.get(include=["documents", "metadatas", "embeddings"])

# ✅ 임베딩
embedding_model = SentenceTransformer("sentence-transformers/all-MiniLM-L12-v2")
query_embedding = embedding_model.encode([query])
similarities = cosine_similarity(query_embedding, results["embeddings"])[0]

# ✅ Top N 유사 문서 보기
top_n = 3
top_indices = similarities.argsort()[-top_n:][::-1]

print(f"🔍 유사도 Top {top_n} 문서:")
for rank, idx in enumerate(top_indices, 1):
    print(f"\n📄 [Rank {rank}] 유사도: {similarities[idx]:.4f}")
    print(f"🆔 ID: {results['ids'][idx]}")
    print(f"📎 출처 URL: {results['metadatas'][idx].get('source_url', '출처 없음')}")
    print(f"📜 문서 내용:\n{results['documents'][idx][:300]}...\n{'-'*80}")

# ✅ 프롬프트 구성 (Top 1 기준)
top_chunk = results["documents"][top_indices[0]]
source_url = results["metadatas"][top_indices[0]].get("source_url", "출처 없음")

prompt = f"""
다음은 장학 관련 내용입니다.:

\"\"\"{top_chunk}\"\"\"

위 내용을 참고하여 다음 질문에 정확히 답변해주세요.

{query}
"""

# ✅ 모델 사용 (사전 로드된 tokenizer, model 사용)
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
outputs = model.generate(**inputs, max_new_tokens=200)
output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

# ✅ 후처리: 질문 반복 제거
answer = output_text.split(query)[-1].strip()  # 질문 뒤에 나온 부분만

# ✅ 최종 출력
print("\n🧠 질문:", query)
print("💡 답변:", answer)
print("🔗 출처 URL:", source_url)


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


🔍 유사도 Top 3 문서:

📄 [Rank 1] 유사도: 0.5935
🆔 ID: 2024학년도 전기(2025년 2월 졸업) 일반대학원 석박사 학위청구논문 심사계획 안내_chunk_2
📎 출처 URL: https://cse.kongju.ac.kr/ZD1110/11543/subview.do?enc=Zm5jdDF8QEB8JTJGYmJzJTJGWkQxMTEwJTJGMTkxNSUyRjMwMDk3NCUyRmFydGNsVmlldy5kbyUzRnBhZ2UlM0QxJTI2c3JjaENvbHVtbiUzRCUyNnNyY2hXcmQlM0QlMjZiYnNDbFNlcSUzRCUyNmJic09wZW5XcmRTZXElM0QlMjZyZ3NCZ25kZVN0ciUzRCUyNnJnc0VuZGRlU3RyJTNEJTI2aXNWaWV3TWluZSUzRGZhbHNlJTI2cGFzc3dvcmQlM0QlMjY%3D
📜 문서 내용:
과정만 해당) ⑨ 논문 유사도 검사 확인서 1부  논문유사도 검색 시스템【카피킬러】이용 방법  【별첨 4 참고】 나. 프로젝트 학위 신청자 ① 프로젝트 학위 대상자 확인원 1부[서식 1-2] ② 프로젝트 학위 청구 심사원 1부[서식 2-2] ③ 프로젝트 지도보고서 1부[서식 3-2] ④ 연구윤리 준수 확인서 1부[서식 4] ⑤ 서약서 1부[서식 5-2] ⑥ 해당과정 성적증명서 1부 ⑦ 심사용 프로젝트 결과보고 서류 각 4부 - 프로젝트 학위 청구 실적물 확인서 4부[서식 6-2] - 한국연구재단 등재(후보)지 이상의 게재 논문(제1...
--------------------------------------------------------------------------------

📄 [Rank 2] 유사도: 0.5753
🆔 ID: 대학원_chunk_0
📎 출처 URL: https://cse.kongju.ac.kr/ZD1110/15507/subview.do
📜 문서 내용:
# 대학원 ## 본문 ○ 교육목표 소프트웨어학과 석·박사과정은 컴퓨터 시스템의 전반적인 원리를 탐구하고 이

In [ ]:
!pip install fastapi uvicorn nest-asyncio pyngrok

In [ ]:
# ✅ 2. FastAPI 실행 준비 ----------------------------여기부터 진성이가 작성함 ---------
import torch
from fastapi import FastAPI, Query
from pydantic import BaseModel
from typing import List
import nest_asyncio
from pyngrok import ngrok
import uvicorn
from fastapi.middleware.cors import CORSMiddleware

app = FastAPI()

# 👇 CORS 설정
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

class RAGResponse(BaseModel):
    answer: str

@app.get("/ask", response_model=RAGResponse)
def ask(user_query: str = Query(..., description="질문을 입력하세요")):
  query = user_query
  inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
  outputs = model.generate(**inputs, max_new_tokens=200)
  answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
  return RAGResponse(answer=answer)

In [ ]:
!ngrok config add-authtoken 2utffVzfm842H2mJG4O3Wrbc9Rm_7zHLoGvQa8UX7J4EYTFXo

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
# ✅ 4. FastAPI 서버 실행 + ngrok으로 외부 연결

nest_asyncio.apply()
public_url = ngrok.connect(8000)
print(f"🚀 외부에서 접속 가능한 API 주소:\n{public_url}/docs")

uvicorn.run(app, host="0.0.0.0", port=8000)

INFO:     Started server process [3018]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


🚀 외부에서 접속 가능한 API 주소:
NgrokTunnel: "https://27dc-34-16-193-54.ngrok-free.app" -> "http://localhost:8000"/docs


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


INFO:     211.119.150.213:0 - "GET /ask?user_query=%EC%86%8C%ED%94%84%ED%8A%B8%EC%9B%A8%EC%96%B4%ED%95%99%EA%B3%BC%20%EC%A1%B8%EC%97%85%20%EB%B0%A9%EB%B2%95 HTTP/1.1" 200 OK
INFO:     27.0.238.181:0 - "GET / HTTP/1.1" 404 Not Found
INFO:     27.0.238.118:0 - "GET / HTTP/1.1" 404 Not Found
INFO:     27.0.238.181:0 - "GET / HTTP/1.1" 404 Not Found
INFO:     27.0.238.181:0 - "GET / HTTP/1.1" 404 Not Found
INFO:     27.0.238.118:0 - "GET / HTTP/1.1" 404 Not Found
INFO:     27.0.238.181:0 - "GET / HTTP/1.1" 404 Not Found
INFO:     211.119.150.213:0 - "GET / HTTP/1.1" 404 Not Found
INFO:     211.119.150.213:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     211.119.150.213:0 - "GET / HTTP/1.1" 404 Not Found


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


INFO:     211.119.150.213:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     211.119.150.213:0 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     211.119.150.213:0 - "GET /ask?user_query=%EC%86%8C%ED%94%84%ED%8A%B8%EC%9B%A8%EC%96%B4%ED%95%99%EA%B3%BC%20%EC%A1%B8%EC%97%85%20%EB%B0%A9%EB%B2%95 HTTP/1.1" 200 OK
INFO:     211.119.150.213:0 - "OPTIONS /ask?user_query=%EC%86%8C%ED%94%84%ED%8A%B8%EC%9B%A8%EC%96%B4%ED%95%99%EA%B3%BC+%EC%A1%B8%EC%97%85+%EB%B0%A9%EB%B2%95 HTTP/1.1" 200 OK
INFO:     211.119.150.213:0 - "OPTIONS /ask?user_query=%EC%86%8C%ED%94%84%ED%8A%B8%EC%9B%A8%EC%96%B4%ED%95%99%EA%B3%BC+%EC%A1%B8%EC%97%85+%EB%B0%A9%EB%B2%95 HTTP/1.1" 200 OK


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


INFO:     211.119.150.213:0 - "GET /ask?user_query=%EC%86%8C%ED%94%84%ED%8A%B8%EC%9B%A8%EC%96%B4%ED%95%99%EA%B3%BC+%EC%A1%B8%EC%97%85+%EB%B0%A9%EB%B2%95 HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [3018]
